In [1]:
import mdtraj as md
import numpy as np
import nglview as nv
from rdkit import Chem
from rdkit.Chem import AllChem

from openmm import app
from openmm import unit
import simtk.openmm as mm
from simtk.openmm.app import Modeller, ForceField, PDBFile, Simulation
import parmed as pmd

from openmmforcefields.generators import SMIRNOFFTemplateGenerator

import openff.toolkit
from openff.toolkit.typing.engines.smirnoff import ForceField as openffForcefield
from openff.toolkit.topology import Molecule, Topology

## Extract a protein ligand complex from 1QKU

In [ ]:
traj = md.load("../1_system_inspection/1QKU.pdb")
topology = traj.topology
print(traj)
print(topology)

In [ ]:
traj = traj.atom_slice(topology.select("not water"))
topology = traj.topology

protein_indices = topology.select("chainid == 0")
ligand_indices = topology.select("chainid == 3")
protein_ligand_indices = np.concatenate((protein_indices, ligand_indices), axis=0)

protein_ligand_complex = traj.atom_slice(protein_ligand_indices)

In [ ]:
view = nv.show_mdtraj(protein_ligand_complex)
view

In [ ]:
#protein_ligand_complex.save_pdb("./protein-ligand.pdb")

## Load ligand and protein
Load the fixed receptor and ligand from previous notebooks 

In [ ]:
ligand_file = "../3_ligand_preparation/ligand.sdf"
protein_file = "../2_receptor_preparation/fixed_receptor.pdb"

In [ ]:
ligand = Molecule.from_file(ligand_file)
ligand_positions = ligand.conformers[0]
ligand_topology = ligand.to_topology()

protein = PDBFile(protein_file)

## Load Forcefield

In [ ]:
forcefield = ForceField("amber99sb.xml", "tip3p.xml")
# Teach OpenMM about the ligand molecule and the Parsley force field
smirnoff = SMIRNOFFTemplateGenerator(forcefield="openff-1.3.0.offxml", molecules=[ligand])
forcefield.registerTemplateGenerator(smirnoff.generator)

## Add solvent

In [ ]:
modeller = Modeller(protein.topology, protein.positions)
modeller.add(ligand_topology.to_openmm(), ligand_positions)

In [ ]:
max_size = max(max((pos[i] for pos in modeller.positions))-min((pos[i] for pos in modeller.positions)) for i in range(3))
vectors = mm.Vec3(1.0, 0, 0), mm.Vec3(1.0/3.0, 2.0*np.sqrt(2.0)/3.0,0.0), mm.Vec3(-1.0/3.0, np.sqrt(2.0)/3.0, np.sqrt(6.0)/3.0)
box_vectors = [(max_size + 14*unit.angstroms)*v for v in vectors]

In [ ]:
# modeller.addSolvent(forcefield, model='tip3p', padding=14*unit.angstroms, neutralize=True)
modeller.addSolvent(forcefield, model='tip3p', boxVectors = box_vectors, neutralize=True)

In [ ]:
# Verify that water was added
n_waters = 0
for chain in modeller.topology.chains():
    for residue in chain.residues():
        if residue.name=='HOH':
            n_waters += 1

print('n_waters: {}'.format(n_waters))

In [ ]:
topology = modeller.getTopology()
positions = modeller.getPositions()
PDBFile.writeFile(topology, positions, open('complex.pdb', 'w'))

In [ ]:
# Visualize complex with water
view = nv.show_structure_file("complex.pdb")
view.add_licorice(selection="(not protein)") # View water molecules
view

##  Create the system

In [ ]:
system = forcefield.createSystem(
    modeller.topology, 
    nonbondedMethod=app.PeriodicCutoff
)

In [ ]:
system.getNumParticles()

## Run a short simulation

In [ ]:
integrator = mm.LangevinIntegrator(
    300 * unit.kelvin, 1 / unit.picosecond, 0.002 * unit.picoseconds
)

# Combine the topology, system, integrator and initial positions into a simulation
simulation = Simulation(
    topology, system, integrator
)
simulation.context.setPositions(positions)

# Add a reporter to record the structure every 10 steps
nc_reporter = pmd.openmm.NetCDFReporter("trajectory.nc", 10)
simulation.reporters.append(nc_reporter)

In [ ]:
simulation.minimizeEnergy()
minimized_state = simulation.context.getState(
    getPositions=True,
    getEnergy=True,
    getForces=True
)

print(
    "Minimised to", 
    minimized_state.getPotentialEnergy(), 
    "with maximum force", 
    max([np.sqrt(v.x*v.x + v.y*v.y + v.z*v.z) for v in minimized_state.getForces()]),
    minimized_state.getForces().unit.get_symbol()
)

minimized_coords = minimized_state.getPositions()

In [ ]:
simulation.context.setVelocitiesToTemperature(300 * unit.kelvin)
simulation.step(1000)

In [2]:
nc_reporter.finalize()

In [3]:
traj = md.load("trajectory.nc", top=md.Topology.from_openmm(topology))

view = nv.show_mdtraj(traj)
view

NameError: name 'topology' is not defined